In [1]:
import torch

In [2]:
x = torch.arange(12)
x

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [3]:
x.shape

torch.Size([12])

In [4]:
x.numel()

12

In [5]:
y = x.reshape(3, 4)
y

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])

In [6]:
torch.zeros(2, 3, 4)

tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])

In [7]:
torch.ones(2, 3, 4)

tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]])

In [8]:
torch.tensor([[0, 1, 2]])

tensor([[0, 1, 2]])

In [9]:
X = torch.arange(12, dtype=torch.float32).reshape(3, 4)
Y = torch.tensor([[2.0, 1, 4, 3], [1, 2, 3, 4], [4, 3, 2, 1]])
torch.cat((X, Y), dim=0), torch.cat((X, Y), dim=1)

(tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [ 2.,  1.,  4.,  3.],
         [ 1.,  2.,  3.,  4.],
         [ 4.,  3.,  2.,  1.]]),
 tensor([[ 0.,  1.,  2.,  3.,  2.,  1.,  4.,  3.],
         [ 4.,  5.,  6.,  7.,  1.,  2.,  3.,  4.],
         [ 8.,  9., 10., 11.,  4.,  3.,  2.,  1.]]))

In [10]:
X.sum()

tensor(66.)

# 广播机制

In [11]:
a = torch.arange(3).reshape(3, 1)
b = torch.arange(2).reshape(1, 2)
a, b

(tensor([[0],
         [1],
         [2]]),
 tensor([[0, 1]]))

In [12]:
a + b  # a、b复制为相同的大小(3, 2)

tensor([[0, 1],
        [1, 2],
        [2, 3]])

In [13]:
X[:-1]

tensor([[0., 1., 2., 3.],
        [4., 5., 6., 7.]])

# 数据预处理

In [14]:
import os
os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n')  # 列名
    f.write('NA,Pave,127500\n')  # 每行表示一个数据样本
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

In [15]:
import pandas as pd

data = pd.read_csv(data_file,header=0)  # 不指定header时默认为0行为header
print(data)

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000


In [16]:
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]
inputs = inputs.fillna(inputs.mean())
inputs

,NumRooms,Alley
0,3.0,Pave
1,2.0,NaN
2,4.0,NaN
3,3.0,NaN


In [17]:
# 将NaN看作一个类别，填充为离散数字
inputs = pd.get_dummies(inputs, dummy_na=True)  # 将分类转换为虚拟变量，也即独热编码
inputs

,NumRooms,Alley_Pave,Alley_nan
0,3.0,1,0
1,2.0,0,1
2,4.0,0,1
3,3.0,0,1


In [18]:
X, y = torch.tensor(inputs.values), torch.tensor(outputs.values)
X, y

(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500, 106000, 178100, 140000]))

In [19]:
X = torch.arange(12,dtype=torch.float32).reshape(3, 4)
X

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.]])

In [20]:
X.sum(), X.sum(axis=0), X.sum(dim=0)

(tensor(66.), tensor([12., 15., 18., 21.]), tensor([12., 15., 18., 21.]))

In [21]:
# 求平均值
X_mean1 = X.sum() / X.numel()
X_mean2 = X.mean()
X_mean3 = X.sum(axis=0) / X.shape[0]
X_mean4 = X.mean(axis=0)
X_mean1,X_mean2,X_mean3,X_mean4

(tensor(5.5000),
 tensor(5.5000),
 tensor([4., 5., 6., 7.]),
 tensor([4., 5., 6., 7.]))

In [22]:
# 保留维度求和
sum_X = X.sum(dim=1,keepdim=True)
sum_X,sum_X.shape,X.shape

(tensor([[ 6.],
         [22.],
         [38.]]),
 torch.Size([3, 1]),
 torch.Size([3, 4]))

In [23]:
cumsum_X = X.cumsum(dim=0)  # 0维度方向累计求和
cumsum_X

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  6.,  8., 10.],
        [12., 15., 18., 21.]])

In [24]:
# numpy中dot直接计算矩阵积，
# torch中dot只能计算向量-向量，矩阵-向量积需要使用mv,矩阵-矩阵使用mm
import numpy as np
Y = torch.arange(0,12,dtype=torch.float32).reshape(4, 3)
X,Y,np.dot(X,Y),torch.mm(X, Y)

(tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.]]),
 tensor([[ 0.,  1.,  2.],
         [ 3.,  4.,  5.],
         [ 6.,  7.,  8.],
         [ 9., 10., 11.]]),
 array([[ 42.,  48.,  54.],
        [114., 136., 158.],
        [186., 224., 262.]], dtype=float32),
 tensor([[ 42.,  48.,  54.],
         [114., 136., 158.],
         [186., 224., 262.]]))

In [27]:
a = torch.arange(0, 12, dtype=torch.float32).reshape(2, 2, 3)
a,torch.norm(a)

(tensor([[[ 0.,  1.,  2.],
          [ 3.,  4.,  5.]],
 
         [[ 6.,  7.,  8.],
          [ 9., 10., 11.]]]),
 tensor(22.4944))